In [1]:
import sys
sys.path.append("../script/")

import Functions
from Engine import Engine
from GTGP import GTGP


from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import importlib
from time import time
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import pandas as pd
from time import time

In [2]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """
    
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    print(roc_auc_score(true_y,prob[:,1]))

# data

In [4]:
df = pd.read_csv("../data/labor.tsv",delimiter='\t')
X = df.iloc[:,:-1].to_numpy().astype("float")
y = df.iloc[:,-1].to_numpy().astype("int")

seeds = [10086, 200,500,30506,30405,30420,10056,7059,40965,5398,869543,83491,823190,
         48392,2810,48392,3498210,483902,859032,12890,538920,86954,54309,6504,9840,
         219805,548,2981,432890,5438908,219094,5843902,60854,979,12890,2108,4093]
train_size = 0.7
dataset = 'labor'

In [5]:
def to_one_hot(y,yt):
    le = LabelEncoder()
    ohe = OneHotEncoder()
    y_train = le.fit_transform(y)
    y_one_hot = ohe.fit_transform(y_train.reshape(-1,1))
    
    y_test = le.transform(yt)
    yt_one_hot = ohe.transform(y_test.reshape(-1,1))
    
    return y_train,y_test,y_one_hot,yt_one_hot

# model

In [24]:
def fit_trees():
    learning_rate=0.1
    max_depth=3
    bins=8
    lam=100

    gtgp = GTGP(learning_rate=learning_rate,max_depth=max_depth,bins=bins,lam=lam)

    total_size=10
    elite_size = 10
    epoch= 100
    gp_epoch= 1
    verbose = 1
    tolerance=0.001

    gtgp.fit(X_train,y_train,total_size=total_size,elite_size = elite_size,epoch=epoch,gp_epoch=gp_epoch,tolerance=tolerance,verbose=verbose)

    retrain_epoch= 20
    alpha=0
    beta=1
    gammer=0

    verbose=1
    gtgp.lam = 10
    gtgp.retrain_estimators(X_test,y_test,retrain_epoch=retrain_epoch,alpha=alpha,beta=beta,gammer=gammer,verbose=verbose)

    return gtgp

for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
    gtgp = fit_trees()

    print(classification_report(y_train,np.argmax(gtgp.train_p,axis=1),digits=5))
    print(classification_report(y_test,np.argmax(gtgp.test_p,axis=1),digits=5))

    num_trees,depth,num_nodes = gtgp.print_model()

    train_acc = accuracy_score(y_train,np.argmax(gtgp.train_p,axis=1))
    test_acc = accuracy_score(y_test,np.argmax(gtgp.test_p,axis=1))
    # train_f1 = f1_score(y_train,np.argmax(gtgp.train_p,axis=1),average='macro')
    # test_f1 = f1_score(y_test,np.argmax(gtgp.test_p,axis=1),average='macro')

    # train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),(gtgp.train_p.T/np.sum(gtgp.train_p,axis=1)).T)
    # test_f1 = roc_auc_score(y_test_one_hot.toarray(),(gtgp.test_p.T/np.sum(gtgp.test_p,axis=1)).T)

    train_f1 = roc_auc_score(gtgp.y_one_hot.toarray(),gtgp.train_p)
    test_f1 = roc_auc_score(y_test_one_hot.toarray(),gtgp.test_p)


    with open('./benchmark/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

10 30 1
19 57 2
28 84 3
37 111 4
46 138 5
54 162 6
64 192 7
73 219 8
82 246 9
90 270 10
98 294 11
105 315 12
114 342 13
122 366 14
131 393 15
140 420 16
145 435 17
152 456 18
159 477 19
168 504 20
172 516 21
177 531 22
185 555 23
190 570 24
196 588 25
202 606 26
209 627 27
215 645 28
222 666 29
227 681 30
229 687 31
235 705 32
240 720 33
244 732 34
249 747 35
254 762 36
259 777 37
265 795 38
272 816 39
276 828 40
282 846 41
286 858 42
289 867 43
292 876 44
295 885 45
303 909 46
310 930 47
311 933 48
315 945 49
318 954 50
324 972 51
326 978 52
330 990 53
331 993 54
335 1005 55
339 1017 56
343 1029 57
347 1041 58
353 1059 59
356 1068 60
357 1071 61
359 1077 62
364 1092 63
365 1095 64
368 1104 65
371 1113 66
374 1122 67
376 1128 68
377 1131 69
378 1134 70
380 1140 71
382 1146 72
384 1152 73
385 1155 74
388 1164 75
390 1170 76
391 1173 77
392 1176 78
395 1185 79
396 1188 80
398 1194 81
400 1200 82
403 1209 83
406 1218 84
406 1218 85
409 1227 86
409 1227 87
410 1230 88
412 1236 89
414 1242 

# Decision Tree

In [25]:
from sklearn.tree import DecisionTreeClassifier

In [26]:
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
    clf = DecisionTreeClassifier()
    clf.fit(X_train,y_train)
    y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)

    num_trees = 1
    depth = clf.tree_.max_depth
    num_nodes = clf.tree_.node_count

    train_acc = accuracy_score(y_train,clf.predict(X_train))
    test_acc = accuracy_score(y_test,clf.predict(X_test))
    train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
    test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')

    train_roc = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
    test_roc = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))

    with open('./benchmark_DC/'+dataset+'.csv','a') as f:
        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# XGBoost

In [27]:
import xgboost

In [28]:
with open('./benchmark_xgb/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # xgb = xgboost.XGBClassifier(n_estimators=100,max_depth=6)
        # xgb = xgboost.XGBClassifier(min_child_weight=0,max_depth=4)
        xgb = xgboost.XGBClassifier(n_estimators=1000)
        xgb.fit(X_train,y_train)

        import json

        def item_generator(json_input, lookup_key):
            if isinstance(json_input, dict):
                for k, v in json_input.items():
                    if k == lookup_key:
                        yield v
                    else:
                        yield from item_generator(v, lookup_key)
            elif isinstance(json_input, list):
                for item in json_input:
                    yield from item_generator(item, lookup_key)

        def tree_depth(json_text):
            json_input = json.loads(json_text)
            depths = list(item_generator(json_input, 'depth'))
            return max(depths) + 1 if len(depths) != 0 else 1

        train_acc = accuracy_score(y_train,xgb.predict(X_train))
        test_acc = accuracy_score(y_test,xgb.predict(X_test))

        # train_f1 = f1_score(y_train,xgb.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,xgb.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),xgb.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),xgb.predict_proba(X_test))

        booster = xgb.get_booster()

        tree_df = booster.trees_to_dataframe()
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        num_trees = len(depths)
        depth = np.average(depths)
        num_nodes = len(tree_df)

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# GDBT

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

In [30]:
with open('./benchmark_GBDT/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # clf = GradientBoostingClassifier(n_estimators=1000)
        clf = GradientBoostingClassifier(n_estimators=1000)
        clf.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,clf.predict(X_train))
        test_acc = accuracy_score(y_test,clf.predict(X_test))
        # train_f1 = f1_score(y_train,clf.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,clf.predict(X_test),average='macro')
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),clf.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),clf.predict_proba(X_test))


        num_trees = len([ est for ests in clf.estimators_ for est in ests])
        depth = np.average([ max(1,est.tree_.max_depth) for ests in clf.estimators_ for est in ests])
        num_nodes = sum([ est.tree_.node_count for ests in clf.estimators_ for est in ests])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)

# Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
with open('./benchmark_RF/'+dataset+'.csv','w') as f:
    for i in range(30):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size,stratify=y, random_state=seeds[i])
        y_train,y_test,y_train_one_hot,y_test_one_hot = to_one_hot(y_train,y_test)
        # rfc = RandomForestClassifier(n_estimators=100)
        rfc = RandomForestClassifier(n_estimators=1000)
        rfc.fit(X_train,y_train)

        train_acc = accuracy_score(y_train,rfc.predict(X_train))
        test_acc = accuracy_score(y_test,rfc.predict(X_test))
        # train_f1 = f1_score(y_train,rfc.predict(X_train),average='macro')
        # test_f1 = f1_score(y_test,rfc.predict(X_test),average='macro')
        
        train_f1 = roc_auc_score(y_train_one_hot.toarray(),rfc.predict_proba(X_train))
        test_f1 = roc_auc_score(y_test_one_hot.toarray(),rfc.predict_proba(X_test))

        num_trees = len(rfc.estimators_)
        depth = np.average([est.tree_.max_depth for est in rfc.estimators_])
        num_nodes = sum([est.tree_.node_count for est in rfc.estimators_])

        s = str(train_acc)+","+str(test_acc)+","+str(train_f1)+","+str(test_f1)+","+str(num_trees)+","+str(depth)+","+str(num_nodes)+"\n"
        f.writelines(s)